# Pull data from each year together and clean/organize it so that I can perform analysis

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
import datetime

### 2018 Data 

In [2]:
# open all 2018 csv files and read them into a dataframe 
files = glob.glob('data/2018/*.csv')

phx_ven_pay_18 = pd.concat([pd.read_csv(f) for f in files], sort=True)

In [3]:
phx_ven_pay_18.shape

(554526, 5)

In [4]:
phx_ven_pay_18.head()

,Check/Payment Date,Commitmt Item Name,Dept. Descrptn,Invoice Net Amt,Vendor Name
0,4/25/2018,Spec Contractual Svc,Human Services,1166.67,1 N 10 INC
1,4/25/2018,Spec Contractual Svc,Human Services,4166.66,1 N 10 INC
2,4/2/2018,Housing Appliances,Housing,1322.86,1 STOP ELECTRONICS CENTER INC
3,4/2/2018,Housing Appliances,Parks and Recreation,1316.38,1 STOP ELECTRONICS CENTER INC
4,4/2/2018,Housing Appliances,Housing,661.43,1 STOP ELECTRONICS CENTER INC


### 2017 Data

In [5]:
# open all 2017 csv files and read them into a dataframe 
files = glob.glob('data/2017/*.csv')

phx_ven_pay_17 = pd.concat([pd.read_csv(f) for f in files], sort=True)

In [6]:
# for whatever reason, this year has a larger number of columns than other years
phx_ven_pay_17.head()

,Invoice Net Amt,Check/Payment Date,Commitmt Item Name,Dept. Descrptn,Document Nbr,Fund Center,Invoice Net Amt,Vendor ID Number,Vendor Name
0,NaN,12/13/2017,Inventories-Offline,Public Works,NaN,NaN,138.18,NaN,1 STOP ELECTRONICS CENTER INC
1,NaN,12/29/2017,Com Software<Cap Lmt,Planning and Development,NaN,NaN,4001.92,NaN,20-20 TECHNOLOGIES
2,NaN,12/29/2017,Com Software<Cap Lmt,Planning and Development,NaN,NaN,-316.92,NaN,20-20 TECHNOLOGIES
3,NaN,12/29/2017,Com Software<Cap Lmt,Planning and Development,NaN,NaN,2122.04,NaN,20-20 TECHNOLOGIES
4,NaN,12/29/2017,Com Software<Cap Lmt,Planning and Development,NaN,NaN,-168.04,NaN,20-20 TECHNOLOGIES


### To clean up:
#### Drop:
+ ~~'Invoice Net Amt' shows up twice, for each column that has a value, the other column has NaN, how can I stitch these together into one column?~~
+ ~~'Vendor ID Number' isn't included in prior and later years, drop this~~
+ ~~'Fund Center' not included in prior and later years, drop this~~
+ ~~'Document Nbr' not included in prior and later years, drop this~~
+ ~~remove three records that don't have Vendor Name~~

In [7]:
# combine two 'Invoice Net Amt' columns into one based on NaN values
phx_ven_pay_17.iloc[:,0].fillna(phx_ven_pay_17.iloc[:,6], inplace=True)

In [8]:
# delete second 'Invoice Net Amt' column since invoice amounts are now being held in first 'Invoice Net Amt' column
phx_ven_pay_17.drop(phx_ven_pay_17.columns[6], axis=1, inplace=True)

In [9]:
# drop unnecessary columns
excess_columns = [
                  'Document Nbr',
                  'Fund Center',
                  'Vendor ID Number']

if set(excess_columns).issubset(phx_ven_pay_17.columns):
   phx_ven_pay_17.drop([ 
                        'Document Nbr', 
                        'Fund Center', 
                        'Vendor ID Number'], 
                       axis=1, inplace=True)

# columns in oldest data = Amount, Date, Department, Description, Vendor Display

In [10]:
# remove handful of rows that have no vendor name
phx_ven_pay_17.dropna(inplace=True)

In [11]:
# remove leading and trailing spaces before and after 'Invoice Net Amt' column label
phx_ven_pay_17.columns = phx_ven_pay_17.columns.str.strip()

In [12]:
phx_ven_pay_17.shape

(487373, 5)

In [13]:
phx_ven_pay_17.dtypes

Invoice Net Amt       object
Check/Payment Date    object
Commitmt Item Name    object
Dept. Descrptn        object
Vendor Name           object
dtype: object

### 2016 Data

In [14]:
files = [
    'data/2016/citycheckbookjantojune2016.csv',
    'data/2016/citycheckbookjulytodec2016.csv',
]

phx_ven_pay_16 = pd.concat([pd.read_csv(f) for f in files], sort=True)

In [15]:
phx_ven_pay_16.head()

,Amount,Date,Department,Description,G/L Description,Vendor Display
0,$855,01-04-2016,Community & Economic Development,Other Commodities,NaN,ABM PARKING SERVICES
1,$471,01-04-2016,Community & Economic Development,Other Commodities,NaN,ABM PARKING SERVICES
2,$813,01-04-2016,Community & Economic Development,Other Commodities,NaN,ABM PARKING SERVICES
3,$578.92,01-04-2016,Fire,Plumbing Services,NaN,ABOVE ALL PLUMBING SERVICES INC
4,"$2,712.04",01-04-2016,Fire,Plumbing Services,NaN,ABOVE ALL PLUMBING SERVICES INC


In [16]:
# rename columns
columns_to_rename = ['Department',
                    'Description',
                    'Amount',
                    'Vendor Display',
                    'Date']

if set(columns_to_rename).issubset(phx_ven_pay_16.columns):
   phx_ven_pay_16.rename(columns={'Department': 'Dept. Descrptn',
                                  'Description': 'Commitmt Item Name',
                                  'Amount': 'Invoice Net Amt',
                                  'Vendor Display': 'Vendor Name',
                                  'Date': 'Check/Payment Date'
                                  }, inplace=True)

In [17]:
# remove 'G/L Description' column
if 'G/L Description' in phx_ven_pay_16.columns:
    phx_ven_pay_16.drop('G/L Description', axis=1, inplace=True)

In [18]:
phx_ven_pay_16.shape

(533015, 5)

In [19]:
phx_ven_pay_16.dtypes

Invoice Net Amt       object
Check/Payment Date    object
Dept. Descrptn        object
Commitmt Item Name    object
Vendor Name           object
dtype: object

### To clean up:
+ ~~change 'Description' to 'Dept. Descrptn' to be consistent with later years~~

### 2015 Data

In [20]:
files = [
    'data/2015/citycheckbookjantojune2015.csv',
    'data/2015/citycheckbookjulytodec2015.csv',
]

phx_ven_pay_15 = pd.concat([pd.read_csv(f) for f in files], sort=True)

In [21]:
phx_ven_pay_15.head()

,Amount,Date,Department,Description,Vendor Display
0,$80,01-02-2015,Municipal Court,Interpreters/Transl,A FOREIGN LANGUAGE SERVICE CORP
1,"$1,888.85",01-02-2015,Aviation,Small Tools/ Equip,A TO Z EQUIPMENT RENTALS
2,$22.46,01-02-2015,Aviation,Motor Vehicle Parts,A TO Z EQUIPMENT RENTALS
3,"$1,973.6",01-02-2015,Aviation,Small Tools/ Equip,A TO Z EQUIPMENT RENTALS
4,$17.33,01-02-2015,Public Works,Inventories,A-Z LOCK PRODUCTS CO INC


### To clean up:

+ ~~change 'Description' to 'Dept. Descrptn' to be consistent with later years~~
+ ~~change 'Department' to 'Dept. Descrptn' to be consistent with later years~~

In [22]:
# rename columns
columns_to_rename = ['Department',
                    'Description',
                    'Amount',
                    'Vendor Display',
                    'Date']

if set(columns_to_rename).issubset(phx_ven_pay_15.columns):
   phx_ven_pay_15.rename(columns={'Department': 'Dept. Descrptn',
                                  'Description': 'Commitmt Item Name',
                                  'Amount': 'Invoice Net Amt',
                                  'Vendor Display': 'Vendor Name',
                                  'Date': 'Check/Payment Date'
                                  }, inplace=True)

In [23]:
phx_ven_pay_15.dtypes

Invoice Net Amt       object
Check/Payment Date    object
Dept. Descrptn        object
Commitmt Item Name    object
Vendor Name           object
dtype: object

In [24]:
phx_ven_pay_15.shape

(333440, 5)

### 2014 Data

In [25]:
files = [
    'data/2014/citycheckbookjantojune2014.csv',
    'data/2014/citycheckbookjulytodec2014.csv',
]

phx_ven_pay_14 = pd.concat([pd.read_csv(f) for f in files], sort=True)

In [26]:
phx_ven_pay_14.head()

,Amount,Date,Department,Description,Vendor Display
0,-$34.78,01-02-2014,Street Transportation,Inventories,3M COMPANY
1,-$69.55,01-02-2014,Street Transportation,Inventories,3M COMPANY
2,-$77,01-02-2014,Street Transportation,Inventories,3M COMPANY
3,$0.01,01-02-2014,Public Works,Inventories,AGS SAFETY & SUPPLY
4,-$0.79,01-02-2014,Police,Inventories,AMERICAN EUROCOPTER CORPORATION


### To clean up:

+ ~~change 'Description' to 'Dept. Descrptn' to be consistent with later years~~
+ ~~convert 'Date' to datetime from object and rename to 'Check/Payment Date' in order to be consistent with later years~~


In [27]:
# rename columns
columns_to_rename = ['Department',
                    'Description',
                    'Amount',
                    'Vendor Display',
                    'Date']

if set(columns_to_rename).issubset(phx_ven_pay_14.columns):
   phx_ven_pay_14.rename(columns={'Department': 'Dept. Descrptn',
                                  'Description': 'Commitmt Item Name',
                                  'Amount': 'Invoice Net Amt',
                                  'Vendor Display': 'Vendor Name',
                                  'Date': 'Check/Payment Date'
                                  }, inplace=True)

In [28]:
phx_ven_pay_14.dtypes

Invoice Net Amt       object
Check/Payment Date    object
Dept. Descrptn        object
Commitmt Item Name    object
Vendor Name           object
dtype: object

In [29]:
phx_ven_pay_14.shape

(307110, 5)

## Join Years 2014 - 2018 Together

In [30]:
phx_ven_pay_14to18 = pd.concat([phx_ven_pay_14,
                               phx_ven_pay_15,
                               phx_ven_pay_16,
                               phx_ven_pay_17,
                               phx_ven_pay_18],
                              sort=False)

In [31]:
phx_ven_pay_14to18.shape

(2215464, 5)

In [32]:
# examine for null values in each column
phx_ven_pay_14to18.isnull().sum()

Invoice Net Amt            3
Check/Payment Date         4
Dept. Descrptn         20791
Commitmt Item Name    243646
Vendor Name                4
dtype: int64

In [33]:
# drop rows with NaN in colums I feel aren't relevant to the analysis I want to do
phx_ven_pay_14to18.dropna(subset=['Invoice Net Amt', 'Check/Payment Date', 'Vendor Name'], inplace=True)

In [34]:
# verify datatypes of each column
phx_ven_pay_14to18.dtypes

Invoice Net Amt       object
Check/Payment Date    object
Dept. Descrptn        object
Commitmt Item Name    object
Vendor Name           object
dtype: object

## To clean up now that I have all the years together:

#### Invoice Net Amt:  
 + Remove $ signs
 + Remove commas(,)
 + Remove open parenthesis '(' used to signify negative values in some years and replace with '-' sign in order to be consistent
 + Remove close parenthesis ')' used to signify negative values in some years
 + Convert to numeric format to be able to sort and work with in a quantitative way
 
#### Check/Payment Date:
 + Replace dash(-) with forward slash(/) so that all dates are consistent
 + Convert to Datetime format
 
#### All Column Labels:
 + Remove spaces in column labels, replace with underscore
 + Convert column names to all lower case characters

In [35]:
phx_ven_pay_14to18.head()

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name
0,-$34.78,01-02-2014,Street Transportation,Inventories,3M COMPANY
1,-$69.55,01-02-2014,Street Transportation,Inventories,3M COMPANY
2,-$77,01-02-2014,Street Transportation,Inventories,3M COMPANY
3,$0.01,01-02-2014,Public Works,Inventories,AGS SAFETY & SUPPLY
4,-$0.79,01-02-2014,Police,Inventories,AMERICAN EUROCOPTER CORPORATION


## Remove $ sign

In [36]:
phx_ven_pay_14to18.iloc[:,0] = phx_ven_pay_14to18.iloc[:,0].str.replace('$','')

In [37]:
# verify all dollar signs have been removed
phx_ven_pay_14to18[phx_ven_pay_14to18['Invoice Net Amt'].apply(lambda x: True if '$' in str(x) else False)].head(50)

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name


In [38]:
phx_ven_pay_14to18.shape

(2215460, 5)

## Remove comma ,

In [39]:
phx_ven_pay_14to18.iloc[:,0] = phx_ven_pay_14to18.iloc[:,0].str.replace(',','')

In [40]:
# verify all commas have been removed
phx_ven_pay_14to18[phx_ven_pay_14to18['Invoice Net Amt'].apply(lambda x: True if ',' in str(x) else False)].head(50)

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name


## Remove left bracket ( and replace with -

In [41]:
phx_ven_pay_14to18.iloc[:,0] = phx_ven_pay_14to18.iloc[:,0].str.replace('(','-')

In [42]:
# verify all left brackets have been removed
phx_ven_pay_14to18[phx_ven_pay_14to18['Invoice Net Amt'].apply(lambda x: True if '(' in str(x) else False)].head(50)

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name


## Remove right bracket )

In [43]:
phx_ven_pay_14to18.iloc[:,0] = phx_ven_pay_14to18.iloc[:,0].str.replace(')','')

In [44]:
# verify all right brackets have been removed
phx_ven_pay_14to18[phx_ven_pay_14to18['Invoice Net Amt'].apply(lambda x: True if ')' in str(x) else False)].head(50)

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name


## Convert to numeric

In [45]:
phx_ven_pay_14to18.iloc[:,0] = pd.to_numeric(phx_ven_pay_14to18.iloc[:,0], errors='raise')

In [46]:
# verify that sorting of values works properly
phx_ven_pay_14to18.sort_values(by='Invoice Net Amt', ascending=False).head()

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name
45978,55205000.0,6/7/2018,Debt - CIC Lease Purchase,Amt Prv-Lng-Trm Oblg,US BANK TRUST NATIONAL ASSOCIATION
209743,49865000.0,6/19/2017,Debt - CIC Lease Purchase,Amt Prv-Lng-Trm Oblg,US BANK TRUST NATIONAL ASSOCIATION
15883,37145000.0,06-06-2016,Debt,Amt Prv-Lng-Trm Oblg,US BANK TRUST NATIONAL ASSOCIATION
40131,32775000.0,06-09-2015,Debt,Amt Prv-Lng-Trm Oblg,US BANK TRUST NATIONAL ASSOCIATION
83963,25595000.0,06-18-2014,Debt,Lease Pur Acc-Princp,US BANK TRUST NATIONAL ASSOCIATION


## Convert Check/Payment Date to datetime

In [47]:
# replace dash(-) with forward slash(/) so that all dates are consistent
phx_ven_pay_14to18.iloc[:,1] = phx_ven_pay_14to18.iloc[:,1].str.replace('-','/')

In [48]:
# convert Check/Payment Date column to datetime format to be able to perform time series analysis with the data
phx_ven_pay_14to18.iloc[:,1] = pd.to_datetime(phx_ven_pay_14to18.iloc[:,1], format='%m/%d/%Y')

In [49]:
# verify earliest and latest dates fall within expected ranges
print(phx_ven_pay_14to18.iloc[:,1].min())
print(phx_ven_pay_14to18.iloc[:,1].max())

2014-01-02 00:00:00
2018-12-31 00:00:00


In [50]:
phx_ven_pay_14to18.head()

,Invoice Net Amt,Check/Payment Date,Dept. Descrptn,Commitmt Item Name,Vendor Name
0,-34.78,2014-01-02,Street Transportation,Inventories,3M COMPANY
1,-69.55,2014-01-02,Street Transportation,Inventories,3M COMPANY
2,-77.00,2014-01-02,Street Transportation,Inventories,3M COMPANY
3,0.01,2014-01-02,Public Works,Inventories,AGS SAFETY & SUPPLY
4,-0.79,2014-01-02,Police,Inventories,AMERICAN EUROCOPTER CORPORATION


## Format column labels to lower case, replace spaces with underscores

In [51]:
def setcols(phx_ven_pay_14to18, fn=lambda x: x.columns.map('_'.join), cols=None):
    """Sets the column of the data frame to the passed column list.
    """
    if cols:
        phx_ven_pay_14to18.columns = cols
    else:
        phx_ven_pay_14to18.columns = fn(phx_ven_pay_14to18)
    return phx_ven_pay_14to18

phx_ven_pay_14to18 = phx_ven_pay_14to18.pipe(
        setcols,
        fn=lambda x: x.columns.str.lower()
        .str.strip()
        .str.replace(" ", "_"))

## Data wrangling complete, write each year to .csv file

In [52]:
phx_ven_pay_14to18[phx_ven_pay_14to18['check/payment_date'].dt.year == 2014] \
    .to_csv('data/cleaned_data/phoenix_vendor_payments_2014.csv', index=False)

In [53]:
phx_ven_pay_14to18[phx_ven_pay_14to18['check/payment_date'].dt.year == 2015] \
    .to_csv('data/cleaned_data/phoenix_vendor_payments_2015.csv', index=False)

In [54]:
phx_ven_pay_14to18[phx_ven_pay_14to18['check/payment_date'].dt.year == 2016] \
    .to_csv('data/cleaned_data/phoenix_vendor_payments_2016.csv', index=False)

In [55]:
phx_ven_pay_14to18[phx_ven_pay_14to18['check/payment_date'].dt.year == 2017] \
    .to_csv('data/cleaned_data/phoenix_vendor_payments_2017.csv', index=False)

In [59]:
phx_ven_pay_14to18[phx_ven_pay_14to18['check/payment_date'].dt.year == 2018] \
    .to_csv('data/cleaned_data/phoenix_vendor_payments_2018.csv', index=False)